# Branch & Bound and Knapsack Lab

**Objectives**
- Preform the branch and bound algorithm
- Apply branch and bound to the knapsack problem
- Understand the geometry of the branch and bound algorithm

**Brief description:** In this lab, we will try solving an example of a knapsack problem with the branch-and-bound algorithm. We will also see how adding a cutting plane helps in reducing the computation time and effort of the algorithm. Lastly, we will explore the geometry of the branch and bound algorithm.


<font color='blue'> <b>Solutions are shown blue.</b> </font> <br>
<font color='red'> <b>Instuctor comments are shown in red.</b> </font>

In [ ]:
# imports -- don't forget to run this cell
from IPython.display import Image
import pandas as pd
import gilp
from gilp.visualize import feasible_integer_pts
from ortools.linear_solver import pywraplp as OR

## Part 1: Branch and Bound Algorithm

Recall that the branch and bound algorithm (in addition to the simplex method) allows us to solve integer programs. Before applying the branch and bound algorithm to the knapsack problem, we will begin by reviewing some core ideas. Furthermore, we will identify a helpful property that will make branch and bound terminate quicker later in the lab!

**Q:** What are the different ways a node can be fathomed during the branch and bound algorithm? Describe each.

**A:** <font color='blue'> You can fathom by:
- integrality: the optimal solution for the node is integral
- bound: the optimal value of the node is not better than the current incumbent
- infeasibility: the node has no feasible solutions
</font>

**Q:** Suppose you have a maximization integer program and you solve its linear program relaxation. What does the LP-relaxation optimal value tell you about the IP optimal value? What if it is a minimization problem?

**A:** <font color='blue'>The IP optimal value is upper-bounded by the LP-relaxation optimal value in the case of a maximization problem and lower-bounded in the case of a minimization problem.</font>

**Q:** Assume you have a maximization integer program with all integral coefficents in the objective function. Now, suppose you are running the branch and bound algorithm and come across a node with an optimal value of 44.5. The current incumbent is 44. Can you fathom this node? Why or why not?

**A:** <font color='blue'>Yes, you can fathom this node because any node with an integral solution in this node's subtree will have an optimal value $\leq 44$ since the coefficents of the objective function are integral. </font>

**Q:** If the optimal solution to the LP relaxation of the original program is integer, then you have found an optimal solution to your integer program. Explain why this is true.

**A:** <font color='blue'> The optimal solution to the LP relaxation is always just as good or better than the optimal integer solution. If the optimal LP solution is integer, that point must be feasible for the IP and so the optimal values of the two are equivalent.</font>

**Q:** If the LP is infeasible, then the IP is infeasible. Explain why this is true.

**A:** <font color='blue'> The LP is less constrainted than the IP. Hence, adding the additional integrality constraint can not add feasible solutions.</font>

The next questions ask about the following branch and bound tree. If the solution was not integral, the fractional $x_i$ that was used to branch is given. If the solution was integral, it is denoted *INT*. In the current iteration of branch and bound, you are looking at the node with the *****.

In [ ]:
Image("images-lab/part1_bnb_tree.png", width=500)

**Q:** Can you determine if the integer program this branch and bound tree is for is a minimization or maximixation problem? If so, which is it?

**A:** <font color='blue'> Yes, it is a minimization problem because the optimal value is higher when constraints are added. </font>

Hint: For **Q7-8**, you can assume integral coefficents in the objective function.

**Q:** Is the current node (marked $z^*$) fathomed? Why or why not? If not, what additional constraints should be imposed for each of the next two nodes?

**A:** <font color='blue'>No, the incumbent is 20 and 16.3 is better. The left branch node should have $x_1 \leq 0$ and the right branch node should have $x_1 \geq 1$</font>

**Q:** Consider the nodes under the current node (where $z = 16.3$). What do you know about the optimal value of these nodes? Why? 

**A:** <font color='blue'> Their objective value must be $\geq 16.3$ because adding a constraint can only make the objective value worse and this is a minimization. </font>

## Part 2: The Knapsack Problem

In this lab, you will solve an integer program by branch and bound. The integer program to be solved will be a knapsack problem.

**Knapsack Problem:** We are given a collection of $n$ items, where each item $i = 1,\dots,n$ has a weight $w_i$ and a value $v_i$. In addition, there is a given capacity $W$, and the aim is to select a maximum value subset of items that has a total weight at most $W$. Note that each item can be brought at most once.

\begin{align*}
\max \quad & \sum_{i=1}^n v_ix_i\\
\text{s.t.} \quad & \sum_{i=1}^n w_ix_i \leq W \\
& 0 \leq x_i \leq 1, \text{integer}, i = 1,\dots,n
\end{align*}

Consider the following data which we import from a CSV file:

In [ ]:
data = pd.read_csv('data/knapsack_data_1.csv', index_col=0)
data

and $W = 18$. 

**Q:** Are there any items we can remove from our input to simplify this problem? Why? If so, replace `index` with the item number that can be removed in the code below. Hint: how many of each item could we possibly take?

**A:** <font color='blue'> We can not feasibly take any of item 7 so we can remove it from the problem!</font>

In [ ]:
# TODO: uncomment and replace index
# data = data.drop(#index)

### BEGIN SOLUTION
data = data.drop(7)
### END SOLUTION

**Q:** If we remove item 7 from the knapsack, it does not change the optimal solution to the integer program. Explain why.

**A:** <font color='blue'> In the integer program, item 7 is either in the knapsack or not. However, since $w_7 > W$, it can never be feasibly included.</font>

**Q:** Consider removing items $i$ such that $w_i > W$ from a knapsack input. How does the LP relaxation's optimal value change?

**A:** <font color='blue'> By removing these items, some fractional feasible solutions were removed. Hence, it is possible that the LP relaxation's optimal value got worse.</font>

In **Q10-11**, you should have found that removing these items removes feasible solutions from the linear program  but does not change the integer program. This is desirable as the gap between the optimal IP and LP values can become smaller. By adding this step, branch and bound may terminate sooner. 

Recall that a branch and bound node can be fathomed if its bound is no better than the value of the best feasible integer solution found thus far. Hence, it helps to have a good feasible integer solution as quickly as possible (so that we stop needless work). To do this, we can first try to construct a good feasible integer solution by a reasonable heuristic algorithm before starting to run the branch and bound procedure.

In designing a heuristic for the knapsack problem, it is helpful to think about the value per unit weight for each item. We compute this value in the table below.

In [ ]:
data['value per unit weight'] = (data['value'] / data['weight']).round(2)
data

**Q:** Design a reasonable heuristic for the knapsack problem. Note a heuristic aims to find a decent solution to the problem (but is not necessarily optimal).

**A:** <font color='blue'>List the items in order (highest to lowest) in terms of value per unit weight. Move down the list taking the item if it fits in the remaining knapsack capacity. </font>

**Q:** Run your heuristic on the data above to compute a good feasible integer solution. Your heuristic should generate a feasible solution with a value of 64 or better. If it does not, try a different heuristic (or talk to your TA!)

**A:** <font color='blue'> First, take item 1. 8 units left. Can't take items 2 or 3. Take item 4. 1 unit left. Can't take items 5,6, or 7. The value of this solution is 64.</font>

We will now use the branch and bound algorithm to solve this knapsack problem! First, let us define a mathematical model for the linear relaxation of the knapsack problem.

**Q:** Complete the model below.

In [ ]:
def knapsack(items, capacity, integer = False):
    """Model for solving the Knapsack problem.
    
    Args:
        items (pd.DataFrame): A table indexd by items with a column for value and weight
        capcity (int): An integer-capacity for the knapsack
        integer (bool): True if the variables should be integer. False otherwise.
    """
    ITEMS = list(items.index)        # set of items
    v = items.to_dict()['value']     # value for each item 
    w = items.to_dict()['weight']    # weight for each item
    W = capacity                     # capacity of the knapsack
    
    # define model
    m = OR.Solver('knapsack', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {}    
    for i in ITEMS:
        if integer:
            x[i] = m.IntVar(0, 1, 'x_%d' % (i)) 
        else:
            x[i] = m.NumVar(0, 1, 'x_%d' % (i)) 
        
    # define objective function here
    m.Maximize(sum(v[i]*x[i] for i in ITEMS))
    
    # weight must not exceed capacity
    # TODO: write this constraint
    
    ### BEGIN SOLUTION
    m.Add(sum(w[i]*x[i] for i in ITEMS) <= W)
    ### END SOLUTION
    
    return m, x

In [ ]:
# You do not need to do anything with this cell but make sure you run it!
def solve(m):
    m.Solve()
    print('Solution:')
    print('Objective value =', m.Objective().Value())
    for var in m.variables():
        print(var.name(), ':',  var.solution_value())

We can now create a linear relaxation of our knapsack problem. Now, `m` represents our model and `x` represents our decision variables.

In [ ]:
m, x = knapsack(data, 18)

We can use the next line to solve the model and output the solution

In [ ]:
solve(m)

**Q:** How does this optimal value compare to the value you found using the heuristic integer solution?

**A:** <font color='blue'> This optimal value (70) is greater than the heuristic value (64). </font>

**Q:** Should this node be fathomed? If not, what variable should be branched on and what additional constraints should be imposed for each of the next two nodes?


**A:** <font color='blue'> The node should not be fathomed. The variable $x_2$ should be branched on because it is fractional. The left branch should have constraint $x_2 \leq 0$ and the right branch should have constraint $x_2 \geq 1$. </font>

After constructing the linear relaxation model using `knapsack(data1, 18)` we can add additional constraints. For example, we can add the constraint $x_2 \leq 0$ and solve it as follows:

In [ ]:
m, x = knapsack(data, 18)
m.Add(x[2] <= 0)
solve(m)

 **NOTE:** The line `m, x = Knapsack(data1, 18)` resets the model `m` to the LP relaxation. All constraints from branching have to be added each time.

**Q:** Use the following cell to compute the optimal value for the other node you found in **Q16**.

In [ ]:
# TODO: Answer Q17

### BEGIN SOLUTION
m, x = knapsack(data, 18)
m.Add(x[2] >= 1)
solve(m)
### END SOLUTION

**Q:** What was the optimal value? Can this node be fathomed? Why? (Hint: In **Q13**, you found a feasible integer solution with value 64.)

**A:** <font color='blue'> The optimal value was 60. This node can be fathomed because we already have an integer feasible solution with a greater objective value (64)</font>

If we continue running the branch and bound algorithm, we will eventually reach the branch and bound tree below where the $z^*$ indictes the current node we are looking at.

In [ ]:
Image("images-lab/part2_bnb_tree.png", width=700)

**Q:** The node with $z = 64.857$ was fathomed. Why are we allowed to fathom this node? (Hint: think back to **Q3**)

**A:** <font color='blue'> We can fathom this node since the objective function has integral coefficients hence any integral feasible soultion in this branch will have value $\leq 64$ and will be fathomed by bound.</font>

**Q:** Finish running branch and bound to find the optimal integer solution. Use a separate cell for each node you solve and indicate if the node was fathomed with a comment. (Hint: Don't forget to include the constraints further up in the branch and bound tree.)

In [ ]:
# Template
m, x = knapsack(data, 18)
# Add constraints here

solve(m)
# fathomed?

In [ ]:
### BEGIN SOLUTION 
m, x = knapsack(data, 18)
m.Add(x[2] <= 0)
m.Add(x[3] <= 0)
m.Add(x[4] >= 1)
m.Add(x[5] <= 0)
m.Add(x[6] <= 0)
solve(m)
# fathom by bound
### END SOLUTION 

In [ ]:
### BEGIN SOLUTION 
m, x = knapsack(data, 18)
m.Add(x[2] <= 0)
m.Add(x[3] <= 0)
m.Add(x[4] >= 1)
m.Add(x[5] <= 0)
m.Add(x[6] >= 1)
solve(m)
# fathom by bound
### END SOLUTION 

**A:** <font color='blue'> An optimal solution is $x_1 = x_4 = 1$ and $x_i = 0$ otherwise with an optimal value of 64.   </font>

**Q:** How many nodes did you have to explore while running the branch and bound algorithm?

**A:** <font color='blue'> 11 nodes were explored. </font>

In the next section, we will think about additional constraints we can add to make running branch and bound quicker.

## Part 3: Cutting Planes

In general, a cutting plane is an additional constraint we can add to an integer program's linear relaxation that removes feasible linear solutions but does not remove any integer feasible solutions. This is very useful when  solving integer programs! Recall many of the problems we have learned in class have something we call the "integrality property". This is useful because it allows us to ignore the integrality constraint since we are garunteed to reach an integral solution. By cleverly adding cutting planes, we strive to remove feasible linear solutions (without removing any integer feasible solutions) such that the optimal solution to the linear relaxation is integral!

Conisder an integer program whose linear program releaxation is 

\begin{align*}
\max \quad & 2x_1+x_2\\
\text{s.t.} \quad & x_1 + x_2 \leq 3 \\
& 2x_1 \leq 5 \\
& -x_1 + 2x_2 \leq 2 \\
& x_1, x_2 \geq 0
\end{align*}

We can define this linear program and then visualize its feasible region. The integer points have been highlighted.

In [ ]:
lp = gilp.LP([[1,1],[2,0],[-1,2]],
             [3,5,2],
             [2,1])
fig = gilp.lp_visual(lp)
fig.set_axis_limits([3.5,2])
fig.add_trace(feasible_integer_pts(lp, fig))
fig

**Q:** List every feasible solution to the integer program.

**A:** <font color='blue'> The feasible solutions are: $(0,0), (0,1), (1,0), (1,1), (2,0), (2,1)$ </font>

**Q:** Is the constraint $x_2 \leq 1$ a cutting plane? Why? (Hint: Would any feasible integer points become infeasible? What about feasible linear points?)

**A:** <font color='blue'> Yes, it removes feasible solutions to the linear relaxation but no integer feasible solutions.  </font>

Let's add this cutting plane to the LP relaxation!

In [ ]:
lp = gilp.LP([[1,1],[2,0],[-1,2],[0,1]],
             [3,5,2,1],
             [2,1])
fig = gilp.lp_visual(lp)
fig.set_axis_limits([3.5,2])
fig.add_trace(feasible_integer_pts(lp, fig))
fig

**Q:** Is the constraint $x_1 \leq 3$ a cutting plane? Why?

**A:** <font color='blue'> No, it does not remove any feasible solutions. </font>

**Q:** Can you provide another cutting plane? If so, what is it?

**A:** <font color='blue'> Yes, $x_1 \leq 2$ </font>

Let's look at the feasible region after adding the cutting plane from **Q** and one of the possible answers from **Q25**. Notice the optimal solution to the LP relaxation is now integral!

In [ ]:
lp = gilp.LP([[1,1],[2,0],[-1,2],[0,1],[1,0]],
             [3,5,2,1,2],
             [2,1])
fig = gilp.lp_visual(lp)
fig.set_axis_limits([3.5,2])
fig.add_trace(feasible_integer_pts(lp, fig))
fig

Let's try applying what we know about cutting planes to the knapsack problem! Again, recall our input was $W = 18$ and:

In [ ]:
data

**Q:** Look at items 1, 2, and 3. How many of these items can we take simultaneously? Can you write a new constraint to capture this? If so, please provide it.

**A:** <font color='blue'> We can only take 1 of items 1, 2, and 3 since they take up half the capacity of the knapsack. We can write the constraint $x_1 + x_2 + x_3 \leq 1$. </font>

**Q:** Is the constraint you found in **Q26** a cutting plane? If so, provide a feasible solution to the linear program relaxation that is no longer feasible (i.e. a point the constraint *cuts off*).

**A:** <font color='blue'> Yes, no integral feasible solutions are cut but the feasible solution $x = (1,\frac{2}{3}, 0,0,0,0)$ is. </font>

**Q:** Provide another cutting plane involving items 4,5 and 6 for this integer program. Explain how you derived it.

**A:** <font color='blue'> We can only take 2 of the three of them in a feasible integral solution so we can add the constraint $x_4 + x_5 + x_6 \leq 2$. This is a cutting plane because $x=(0,0,0,1,\frac{2}{3},1)$ is no longer feasible. </font>

**Q:** Add the cutting planes from **Q26** and **Q28** to the model and solve it. You should get a solution in which we take items 1 and 4 and $\frac{1}{6}$ of item 5 with an objective value of 66.

In [ ]:
m, x = knapsack(data, 18)
# TODO: Add cutting planes here

### BEGIN SOLUTION
m.Add(x[1] + x[2] + x[3] <= 1)
m.Add(x[4] + x[5] + x[6] <= 2)
### END SOLUTION

solve(m)

Let's take a moment to pause and reflect on what we are doing. Recall from **Q9-11** that we dropped item 7 becuase its weight was greater than the capcity of the knapsack. Essentially we added the constraint $x_7 \leq 0$. This constraint was a cutting plane! It eliminated some linear feasible solutions but no integer ones. By adding these two new cutting planes, we can get branch and bound to terminate earlier yet again! So far, we have generated cutting planes by inspection. However, there are more algorithmic ways to identify them (which we will ignore for now).

If we continue running the branch and bound algorithm, we will eventually reach the branch and bound tree below where the $z^*$ indictes the current node we are looking at.

In [ ]:
Image("images-lab/part3_bnb_tree.png", width=700)

**NOTE:** Do not forget about the feasible integer solution our heuristic gave us with value 64.

**Q** Finish running branch and bound to find the optimal integer solution. Use a separate cell for each node you solve and indicate if the node was fathomed with a comment. Hint: Don't forget the cutting plane constraints should be included in every node of the branch and bound tree.

In [ ]:
# Template
m, x = knapsack(data, 18)
# Add constraints here

solve(m)
# fathomed?

In [ ]:
### BEGIN SOLUTION
m, x = knapsack(data, 18)
m.Add(x[1] + x[2] + x[3] <= 1)
m.Add(x[4] + x[5] + x[6] <= 2)
m.Add(x[5] >= 1)
m.Add(x[4] <= 0)
solve(m)
# fathom by bound
### END SOLUTION

In [ ]:
### BEGIN SOLUTION
m, x = knapsack(data, 18)
m.Add(x[1] + x[2] + x[3] <= 1)
m.Add(x[4] + x[5] + x[6] <= 2)
m.Add(x[5] >= 1)
m.Add(x[4] >= 1)
solve(m)
# fathom by bound
### END SOLUTION

**A:** <font color='blue'> An optimal solution is $x_1 = x_4 = 1$ and $x_i = 0$ otherwise with an optimal value of 64.   </font>

**Q:** Did you find the same optimal solution? How many nodes did you explore? How did this compare to the number you explored previously?

**A:** <font color='blue'> Same optimal solution $x = (1,0,0,1,0,0)$ but only took 7 nodes this time compared to the 11 it took with only one cutting plane.</font>

## Part 4: Geometry of Branch and Bound

Previously, we used the `gilp` package to viusualize the simplex algorithm but it also has the functionality to visualize branch and bound. We will give a quick overview of the tool. Similar to `lp_visual` and `simplex_visual`, the function `bnb_visual` takes an `LP` and returns a visualization. It is assumed that every decision variable is constrained to be integer. Unlike previous visualizations, `bnb_visual` returns a series of figures for each node of the branch and bound tree. Let's look at a small 2D example:

\begin{align*}
\max \quad & 5x_1+ 8x_2\\
\text{s.t.} \quad & x_1 + x_2 \leq 6 \\
& 5x_1 + 9x_2 \leq 45 \\
& x_1, x_2 \geq 0, \quad \text{integral}
\end{align*}

In [ ]:
nodes = gilp.bnb_visual(gilp.examples.STANDARD_2D_IP)

In [ ]:
nodes[0].show()

Run the cells above to generate a figure for each node and view the first node. At first, you will see the LP relaxation on the left and the root of the branch and bound tree on the right. The simplex path and isoprofit slider are also present.

**Q:** Recall the root of a branch and bound tree is the unaltered LP relaxation. What is the optimal solution? (Hint: Use the objective slider and hover over extreme points).

**A:** <font color='blue'> The optimal solution is $x = (2.25, 3.75)$</font>

**Q:** Assume that we always choose the variable with the minimum index to branch on if there are multiple options. Write down (in full) each of the LPs we get after branching off the root node.

**A:** <font color='blue'> \begin{align*}
\max \quad & 5x_1+ 8x_2\\
\text{s.t.} \quad & x_1 + x_2 \leq 6 \\
& 5x_1 + 9x_2 \leq 45 \\
& x_1 \leq 2 \\
& x_1, x_2 \geq 0
\end{align*}

and 

\begin{align*}
\max \quad & 5x_1+ 8x_2\\
\text{s.t.} \quad & x_1 + x_2 \leq 6 \\
& 5x_1 + 9x_2 \leq 45 \\
& x_1 \geq 3 \\
& x_1, x_2 \geq 0
\end{align*}</font>

**Q:** Draw the feasible region to each of the LPs from **Q33** on the same picture.

**A:** <font color='blue'> Should look like the result from `nodes[1].show()` </font>

Run the following cell to see if the picture you drew in **Q** was correct.

In [ ]:
nodes[1].show()

The outline of the original LP relaxation is still shown on the left. Now that we have eliminated some of the fractional feasible solutions, we now have 2 feasible regions to consider. The darker one is the feasible region associated with the current node which is also shaded darker in the branch and bound tree. The unexplored nodes in the branch and bound tree are not shaded in.

**Q:** Which feasible solutions to the LP relaxation are removed by this branch?

**A:** <font color='blue'> Those with $2 < x_1 < 3$.</font>

**Q:** At the current (dark) node, what constraints will we add? How many feasible regions will the original LP relaxation be broken into?

**A:** <font color='blue'> We add the constraints $x_2 \leq 3$ and $x_2 \geq 4$. It is divided into 3 feasible regions. </font>

In [ ]:
nodes[2].show()

**Q:** What is the optimal solution at the current (dark) node? Do we have to further explore this branch? Explain.

**A:** <font color='blue'> The optimal solution is 34. We do not have to further explore this branch since the optimal solution was integral. Hence, it must be an optimal solution to the IP for this node. </font>

**Q:** Recall shaded nodes have been explored and the node shaded darker (and feasible region shaded darker) correspond to the current node and its feasible region. Nodes not shaded have not been explored. How many nodes have not yet been explored?

**A:** <font color='blue'>2</font>

**Q:** How many nodes have a degree of one in the branch and bound tree? (That is, they are only connected to one edge). These nodes are called leaf nodes. What is the relationship between the leaf nodes and the remaining feasible region?

**A:** <font color='blue'>There are 3 leaf nodes and each corresponds to one of the remaining feasible regions. </font>

In [ ]:
# Show the next two iterations of the branch and bound algorithm
nodes[3].show()
nodes[4].show()

**Q:** At the current (dark) node, we added the constraint $x_1 \leq 1$. Why were the fractional solutions $1 < x_1 < 2$ not eliminated for $x_2 <= 3$?

**A:** <font color='blue'> Because we were on a node in the branch and bound tree with the constraint $x_2 \geq 4$ so our branch would not affect fractional solutions associated with a different node in the tree.</font>

In [ ]:
# Show the next three iterations of the branch and bound algorithm
nodes[5].show()
nodes[6].show()
nodes[7].show()

**Q:** What constraints are enforced at the current (dark) node? Why are there no feasible solutions at this node?

**A:** <font color='blue'> In addition to the original constraints, we have $x_1 \leq 2$, $x_2 \geq 4$, and $x_1 \geq 2$. No feasible solutions have $x_1 = 2$ and $x_2 \geq 4$.</font>

In [ ]:
nodes[8].show()

**Q:** Are we done? If so, what nodes are fathomed and what is the optimal solution? Explain.

**A:**  <font color='blue'> Yes! In this case, all the nodes were fathomed by integrality with the exception of the one fathomed by infeasibility. The optimal solution is $x = (0,5)$ with the objective value of 40 because it is the best integral solution we found.</font>

Let's look at branch and bound visualization for an integer program with 3 decision variables!

In [ ]:
nodes = gilp.bnb_visual(gilp.examples.VARIED_BRANCHING_3D_IP)

In [ ]:
# Look at the first 3 iterations
nodes[0].show()
nodes[1].show()
nodes[2].show()

Let's fast-forward to the final iteration of the branch and bound algorithm.

In [ ]:
nodes[-1].show()

**Q:** Consider the feasible region that looks like a rectangular box with one corner point at the origin. What node does it correspond to in the tree? What is the optimal solution at that node?

**A:** <font color='blue'>It is the node with the addtional constraints $x_2 \leq 2$, $x_1 \leq 1$, $x_3 \leq 2$ and it has an optimal solution of $x=(1,2,2)$. </font>

**Q:** How many branch and bound nodes did we explore? What was the optimal solution? How many branch and bound nodes would we have explored if we knew the value of the optimal solution before starting branch and bound?

**A:** <font color='blue'>We explored 13 nodes. The optimal solution was $x = (0,3,1)$ with value 13. Had we known the optimal IP value was 13, we would have only explored 3 nodes.</font>

## Bonus: Branch and Bound for Knapsack

Consider the following example:

| item | value | weight
|------|-------|-------
|  1   |   2   |   1 
|  2   |   9   |   3 
|  3   |   6   |   2 

The linear program formulation will be:


\begin{align*}
\max \quad & 2x_1+9x_2+6x_3\\
\text{s.t.} \quad & 1x_1+3x_2+2x_3 \leq 10 \\
& x_1, x_2, x_3 \geq 0, \quad \text{integer}
\end{align*}

In gilp, we can define this lp as follows:

In [ ]:
lp = gilp.LP([[1,3,2]],
             [10],
             [2,9,6])

for fig in gilp.bnb_visual(lp):
    fig.show()